# Image Classifier with CNN

#### Import libraries for preprocessing data

In [7]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
from PIL import ImageFile
import random
from tqdm import tqdm
from keras.preprocessing import image
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

random.seed(340958234985)
np.random.seed(2093846)

Using TensorFlow backend.


#### Load data from files

In [2]:
def load_dataset(path):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    image_targets = np_utils.to_categorical(np.array(data['target']), 55)
    return image_files, image_targets


image_files, image_targets = load_dataset('images')

trains_validate_files, test_files, trains_validate_targets, test_targets = \
    train_test_split(image_files, image_targets, test_size=0.2, random_state=42)

train_files, valid_files, train_targets, valid_targets = \
    train_test_split(trains_validate_files, trains_validate_targets, test_size=0.25, random_state=42)

image_names = [item[20:-1] for item in sorted(glob("images/*/"))]

print('%d image categories.' % len(image_names))
print('%s images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('%d training images.' % len(train_files))
print('%d validation images.' % len(valid_files))
print('%d test images.'% len(test_files))

55 fruit categories.
38409 fruit images.

23045 training images.
7682 validation images.
7682 test images.


#### Define function for preprocessing images

In [3]:
def path_to_tensor(img_path):

    img = image.load_img(img_path, target_size=(100, 100))
    img_array = image.img_to_array(img)

    return np.expand_dims(img_array, axis=0)

def paths_to_tensor(img_paths):
    
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

#### Preprocess images

In [4]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 7682/7682 [00:10<00:00, 742.38it/s]


#### Define neural network model

In [17]:
model = Sequential()

model.add(Conv2D(filters=4, kernel_size=2, padding='same',
                 activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(filters=12, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(len(image_names), activation='softmax'))


model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               7680256   
_________________________________________________________________
dense_19 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_20 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_21 (Dense)             (None, 55)                14135     
Total params: 7,825,975
Trainable params: 7,825,975
Non-trainable params: 0
_________________________________________________________________


#### Train the model with training and validating images

In [15]:
epochs = 5

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.image_classifier.hdf5',
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=64, callbacks=[checkpointer], verbose=1)

Train on 23045 samples, validate on 7682 samples
Epoch 1/5
23045/23045 [==============================] - 87s 4ms/step - loss: 15.5909 - acc: 0.0168 - val_loss: 15.8328 - val_acc: 0.0177

Epoch 00001: val_loss improved from inf to 15.83275, saving model to saved_models/weights.best.classifruit.hdf5
Epoch 2/5
23045/23045 [==============================] - 83s 4ms/step - loss: 15.8432 - acc: 0.0171 - val_loss: 15.8328 - val_acc: 0.0177

Epoch 00002: val_loss did not improve
Epoch 3/5
23045/23045 [==============================] - 74s 3ms/step - loss: 15.8432 - acc: 0.0171 - val_loss: 15.8328 - val_acc: 0.0177

Epoch 00003: val_loss did not improve
Epoch 4/5
23045/23045 [==============================] - 78s 3ms/step - loss: 15.8432 - acc: 0.0171 - val_loss: 15.8328 - val_acc: 0.0177

Epoch 00004: val_loss did not improve
Epoch 5/5
23045/23045 [==============================] - 71s 3ms/step - loss: 15.8432 - acc: 0.0171 - val_loss: 15.8328 - val_acc: 0.0177

Epoch 00005: val_loss did not 

#### Show result on testing set

In [16]:
model.load_weights('saved_models/weights.best.image_classifier.hdf5')

predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 1.6532%
